<a href="https://colab.research.google.com/github/fphans/BIDS-NGS/blob/main/NGS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
R.version.string
#print(installed.packages())

[1] "R version 4.1.0 (2021-05-18)"

RNA Seq Analyse
Datum: 18.06.2021
Autor: Patrick Metzger

# Introduction

Wir bitten Sie hier, die grundlegenden Schritte einer RNA-Sequenzierungs-Analyse durchzuführen. Wir stellen Ihnen bereits die zum menschlichen Referenzgenom zugeordneten (aligned) Reads zur Verfügung, d.h. Ihr Ausgangspunkt sind die 'Counts' für jede Probe. Zusammen mit den Counts (*_ReadsPerGene.out.tab) stellen wir Ihnen die Annotationstabelle (targets.xlsx) zur Verfügung, die die Proben-IDs mit den tatsächlichen biologischen Bedingungen und "Real-World-Proben" verknüpft. Das Ziel der Studie ist es, den Einfluss einer KLF7-Überexpression zu identifizieren. 

Dazu wurde die Zelllinie so modifiziert, dass sie KLF7 Überexprimiert. Als Kontrolle wurde die unbehandelte Zelllinie verwendet. Die Überexprimierende Bedingung ist mit "*_KLF7" und die Kontrolle mit "*_LV" beschriftet. LV steht hier für "Leervektor".

Ihre Aufgabe ist es nun, die signifikant veränderten Gene (in Bezug auf die Genexpression) zwischen den behandelten Proben (KLF7) und der unbehandelten Kontrolle (LV) zu identifizieren. Dazu muss eine Analyse der differentiell exprimierten Gene (diffenretially expressed genes DEG) durchgeführt werden. Im folgenden Skript geben wir Ihnen dazu einige Hinweise und Hilfestellungen.

# Die folgenden Schritte werden abgedeckt

1. die Zähldaten müssen importiert werden
2. die Proben mit den biologischen Bedingungen verknüpfen
3. eine Meta-Tabelle mit allen gemessenen Genen und allen verfügbaren Gen-Identifikatoren erzeugen
4. eine Hauptkomponentenanalyse (PCA) berechnen, um einen ersten Überblick über den Datensatz zu erhalten
5. führen Sie die eigentliche DEG-Analyse durch. Wie viele Gene sind signifikant verändert bis zu einem korrigiertem p-Wert < 0,05; Wie viele davon sind hoch- bzw. herunterreguliert?
6. visualisieren und exportieren Sie Ihre Ergebnisse
7. erzeugen Sie Heatmaps der hochvariablen Gene

## Nützliche Befehle

head()  
?functionName or help()  
install.packages()  
accessing columns by name, e.g. `result$padj filter data.frame`, e.g. `res[res$test > 5,]` the output contains all columns of res but only the rows fulfilling that the value within the "test" column contains a value bigger than 5  
dim()  
length()  
is.na() / !is.na()  

## Nützliche Links #

links / tutorials / vignettes  
https://www.bioconductor.org/install/  
https://bioconductor.org/packages/release/bioc/html/DESeq2.html  
https://cran.r-project.org/web/packages/FactoMineR/index.html  
https://ggplot2.tidyverse.org ; https://cran.r-project.org/web/packages/ggplot2/index.html  
https://cran.r-project.org/web/packages/ggrepel/vignettes/ggrepel.html  
https://bioconductor.org/packages/release/bioc/html/genefilter.html  

In [67]:
#############
# Libraries #
# used packages
# if not installed please install with either bioconductor BiocManager::install() (see useful links) or install.packages()

if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
BiocManager::install(version = "3.13")
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
BiocManager::install("Pakete")
BiocManager::install("DESeq2")
BiocManager::install(c('pheatmap','openxlsx','org.Hs.eg.db','ggplot2','genefilter','FactoMineR','ggrepel','apeglm','ggrepel','genefilter'))

library(DESeq2)
library(openxlsx)
library(pheatmap)
library(org.Hs.eg.db)
library(ggplot2)
library(genefilter)
library(FactoMineR)
library(ggrepel)
library(apeglm)
library(ggrepel)
library(genefilter)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.13 (BiocManager 1.30.16), R 4.1.0 (2021-05-18)

Old packages: 'broom', 'colorspace', 'curl', 'dplyr', 'gert', 'ggplot2',
  'mime', 'openssl', 'rmarkdown', 'testthat', 'xfun'

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.13 (BiocManager 1.30.16), R 4.1.0 (2021-05-18)

Installing package(s) 'Pakete'

Warning message in .inet_warning(msg):
“package ‘Pakete’ is not available for Bioconductor version '3.13'

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Old packages: 'broom', 'colorspace', 'curl', 'dplyr', 'gert', 'ggplot2',
  'mime', 'openss

In [68]:
############################################
###                                      ###
###               FUNCTIONS              ###
###                                      ###
############################################

# Funktionen um verschiedene Gen-IDs zu konvertieren und das geneMat data.frame zu erstellen, welche alle
# verschiedenen IDs pro Gen enthält, die im Datensatz enthalten sind. Alle Konvertierungen basieren auf der org.Hs.eg.db-Datenbank.
# Führen Sie die folgende Funktion aus, bevor Sie fortfahren

ensembl2entrez <- function(ensembl)
{
  entrez <- mget(as.character(ensembl), org.Hs.egENSEMBL2EG, ifnotfound=NA)
  entrez <- lapply(entrez, function(i) return(i[1]))
  return(unlist(entrez))
}

entrez2ensembl <- function(entrez)
{
  esbl <- mget(as.character(entrez), org.Hs.egENSEMBL, ifnotfound=NA)
  esbl <- lapply(esbl, function(i) return(i[1]))
  return(unlist(esbl))
}

entrez2symbol <- function(entrez)
{
  symbol <- mget(as.character(entrez), org.Hs.egSYMBOL, ifnotfound=NA)
  symbol <- unlist(lapply(symbol, function(i) return(i[1])))
  return(symbol)
}

entrez2genename <- function(entrez)
{
  symbol <- mget(as.character(entrez), org.Hs.egGENENAME, ifnotfound=NA)
  symbol <- unlist(lapply(symbol, function(i) return(i[1])))
  return(symbol)
}

getGeneMat <- function(ensIDs){
  geneMat <- data.frame(ENSEMBL=ensIDs)
  geneMat$ENTREZ <- ensembl2entrez(geneMat$ENSEMBL)
  idxNA <- !is.na(geneMat$ENTREZ)
  sym <- entrez2symbol(na.omit(geneMat$ENTREZ))
  genename <- entrez2genename(na.omit(geneMat$ENTREZ))
  geneMat$Symbol <- NA
  geneMat$Symbol[idxNA] <- sym
  geneMat$Genename <- NA
  geneMat$Genename[idxNA] <- genename
  rownames(geneMat) <- geneMat$ENSEMBL
  return(geneMat)
}

In [ ]:
########
# MAIN #

## Importieren und Erstellen der Count-Matrix zusammen mit der Gen-Matrix
## Create Count Matrix

# Load /import count data
targets <- read.xlsx(xlsxFile = "targets.xlsx")
countFiles <- list.files(pattern = "ReadsPerGene.out.tab")
names(countFiles) <- gsub("_ReadsPerGene.out.tab", "", countFiles)
countList <- lapply(countFiles, read.delim, skip = 4, header = FALSE, stringsAsFactors = FALSE)
names(countList) <- gsub("_ReadsPerGene.out.tab", "", targets$ID)
rownames(targets) <- targets$ID